### Set the corresponding values in the cell below. Afterwards, just run the following cells

In [20]:
import pathlib
# Class label csv path
labels_csv_path = 'datasets/home_labels.csv'
# Datset folder
dataset_root_path = pathlib.Path('datasets')
# output result csv path
output_csv_path = 'outputs/clap_results.csv'

### Import libraries 

In [21]:
from msclap import CLAP
import torch.nn.functional as F
import numpy as np
import torch
import os
from moviepy.editor import VideoFileClip

### Get Audio from video

### Read class Labels

In [22]:
import csv

label2id = {}
id2label = {}
with open(labels_csv_path, mode='r') as file:
    csv_reader = csv.reader(file)

    for i, row in enumerate(csv_reader):
        class_name = row[0]
        label2id[class_name] = i 
        id2label[i] = class_name

class_labels = list(label2id.keys())

print(f"{len(class_labels)} Unique classes: {class_labels}.")

70 Unique classes: ['No people is in the room', 'Studying', 'Typing', 'Using computer', 'Making phone calls', 'Play with phone/tablet', 'Playing with pets', 'Sitting down', 'Reading', 'Writing', 'Setting table', 'Eating food', 'Dining', 'Cooking', 'Sleeping', 'Laying down', 'Picking up objects', 'Brushing teeth', 'Showering', 'Playing music', 'TV', 'Cleaning', 'Doing laundry', 'Mopping floor', 'Vacumning', 'Organizing space', 'Sewing', 'Knitting', 'Decorating', 'Party', 'Chatting', 'Talking', 'Singing', 'Laughing', 'Speaking', 'Dancing', 'Drinking', 'Meditating', 'Drawing', 'Painting', 'Playing board games', 'Playing video games', 'Taking photos', 'Potluck', 'Working', 'Exercising', 'Walking', 'Running', 'Celebrating', 'Physical altercations', 'Verbal confrontations', 'Using drug', 'Theft or vandalism', 'Fighting', 'Domestic violence', 'Break in', 'Glass breaking', 'Unattended cooking', 'Smoke or Fire or Open flame', 'Smoking', 'Gunshot', 'Making noise', 'Falling down', 'Tripping', 'Cr

### CLAP inference with given audio

In [23]:
from tqdm import tqdm
from collections import defaultdict
import csv

train_paths = list(dataset_root_path.glob("train/*/*"))

# Load model (Choose between versions '2022' or '2023')
# The model weight will be downloaded automatically if `model_fp` is not specified
clap_model = CLAP(version = '2023', use_cuda=True)

correct = defaultdict(int)
total = defaultdict(int)
with open(output_csv_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["file path", "ground truth", "top1 pred", "confidence", "top2 pred", "confidence", "top3 pred", "confidence", "top4 pred", "confidence", "top5 pred", "confidence"])

    for path in tqdm(train_paths):
        label = str(path).split('\\')[-2]
        out_row = [path, label2id[label]]
        total[label] += 1

        # sanity format check
        try: assert(str(path).endswith(".mp4") or str(path).endswith(".wav"))
        except: print(f"ERROR: {path} is not a supported file type")

        path = path.as_posix()
        if str(path).endswith(".mp4"):
            video = VideoFileClip(path)
            if video.audio is None:
                print(f"This video {path} has no audio!")
                out_row.extend([-1, 0] * 5) # use -1 if no audio
                writer.writerow(out_row)
                continue
            else:
                audio_path = 'datasets/audios' + path[8:-3] + 'wav'
                audio_dir = "/".join(audio_path.split("/")[:-1])
                print(audio_dir)
                if not os.path.exists(audio_dir):
                        os.makedirs(audio_dir)
                audio_arr = video.audio.write_audiofile(audio_path, verbose=False)
                # print(f"Audio saved to {audio_path}")
        else:
            audio_path = path
        
        # make CLAP predictions
        with torch.no_grad():

            # Extract text embeddings
            text_embeddings = clap_model.get_text_embeddings([f"This is a sound of {c}"for c in class_labels])

            # Extract audio embeddings
            audio_embeddings = clap_model.get_audio_embeddings([audio_path])

            # Compute similarity between audio and text embeddings 
            similarities = clap_model.compute_similarity(audio_embeddings, text_embeddings)

        similarity = F.softmax(similarities, dim=1)
        values, indices = similarity[0].topk(5)
        pred_index = indices[0]

        # Output the results
        
        print("\nFile:", path)
        print("Ground Truth: {}".format(audio_path.split('/')[-2]))
        print("Top predictions:")
        for value, index in zip(values, indices):
            print(f"{class_labels[index]:>16s}: {100 * value.item():.2f}%")
            out_row.append(index.item())
            out_row.append(round(value.item(), 6))
        print("")
        writer.writerow(out_row)

        if label == class_labels[pred_index]: 
            correct[label] += 1
        


  0%|          | 0/422 [00:00<?, ?it/s]

datasets/audios/train/Crying
MoviePy - Writing audio in datasets/audios/train/Crying/9q8mf6GUiiE_000009_000019.wav


  0%|          | 0/422 [00:00<?, ?it/s]

MoviePy - Done.


  0%|          | 1/422 [00:00<02:45,  2.54it/s]

...

This video datasets/train/Smoke or Fire or Open flame/Video78.mp4 has no audio!
This video datasets/train/Smoke or Fire or Open flame/Video82.mp4 has no audio!


100%|█████████▉| 420/422 [01:26<00:00, 11.85it/s]

This video datasets/train/Smoke or Fire or Open flame/Video85.mp4 has no audio!


100%|██████████| 422/422 [01:26<00:00,  4.89it/s]

This video datasets/train/Smoke or Fire or Open flame/Video91.mp4 has no audio!
This video datasets/train/Smoke or Fire or Open flame/Video99.mp4 has no audio!


In [24]:
for label in total:
    print(f"{label}: {correct[label]} / {total[label]}")

Crying: 36 / 50
Falling down: 0 / 30
Glass breaking: 10 / 52
Gunshot: 0 / 50
Normal: 0 / 200
Smoke or Fire or Open flame: 0 / 40
